<a href="https://colab.research.google.com/github/Niveditha-hd/ERA_V2/blob/main/model_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount to the google drive
from google.colab import drive
drive.mount('/content/gdrive')
import sys
sys.path.append('/content/gdrive/MyDrive/Colab Notebooks')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

CODE BLOCK: 7

In [ ]:
class Net(nn.Module):
  # This neural network has 4 layers of convolution
    def __init__(self):
        super(Net, self).__init__()
        #input channels = 1, #output channels = 32
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        #input channels = 32, #output channels = 64
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        #input channels = 64, #output channels = 128
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3)
         #input channels = 128, #output channels = 256
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3)
        self.fc1 = nn.Linear(4096, 50)
        self.fc2 = nn.Linear(50, 10)
# forward function defines the actual behaviour of the desired function
# Gray scale image of 28*28 is the input
# After every 2 layers, maxpool function is performed.
    def forward(self, x):
        x = F.relu(self.conv1(x), 2) # Channel: 28>26 | RF: 1>3
        x = F.relu(F.max_pool2d(self.conv2(x), 2)) #26>24(conv2)>12 (maxpool)|RF: 3>5>6
        x = F.relu(self.conv3(x), 2) # 12>10 | RF: 6>10
        x = F.relu(F.max_pool2d(self.conv4(x), 2)) # 10>8(conv4)>4 (maxpool)| RF: 10>14>16
        x = x.view(-1, 4096) # (4*4)*256 = 4096
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

CODE BLOCK: 9

In [ ]:

from tqdm import tqdm

#
def GetCorrectPredCount(pPrediction, pLabels):
  return pPrediction.argmax(dim=1).eq(pLabels).sum().item()

# Supply train data, model and train
def train(model, device, train_loader, optimizer, criterion):
  model.train()
  pbar = tqdm(train_loader)

  train_loss = 0
  correct = 0
  processed = 0

  for batch_idx, (data, target) in enumerate(pbar):
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()

    # Predict
    pred = model(data)

    # Calculate loss
    loss = criterion(pred, target)
    train_loss+=loss.item()

    # Backpropagation
    loss.backward()
    optimizer.step()

    correct += GetCorrectPredCount(pred, target)
    processed += len(data)

    pbar.set_description(desc= f'Train: Loss={loss.item():0.4f} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')

  train_acc.append(100*correct/processed)
  train_losses.append(train_loss/len(train_loader))

def test(model, device, test_loader, criterion):
    model.eval()

    test_loss = 0
    correct = 0

    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)

            output = model(data)
            test_loss += criterion(output, target, reduction='sum').item()  # sum up batch loss

            correct += GetCorrectPredCount(output, target)


    test_loss /= len(test_loader.dataset)
    test_acc.append(100. * correct / len(test_loader.dataset))
    test_losses.append(test_loss)

    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
